In [3]:
#Here I downloaded the T5 Pre trained model
from transformers import T5Tokenizer, T5ForConditionalGeneration
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/pyth

In [4]:
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, tokenizer, texts, targets, max_len=512):
        self.tokenizer = tokenizer
        self.texts = texts
        self.targets = targets
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        input_text = self.texts[index]
        target_text = self.targets[index]

        input_encoding = self.tokenizer(input_text, max_length=self.max_len, padding='max_length', truncation=True, return_tensors='pt')
        target_encoding = self.tokenizer(target_text, max_length=self.max_len, padding='max_length', truncation=True, return_tensors='pt')

        return {
            'input_ids': input_encoding['input_ids'].squeeze(),
            'attention_mask': input_encoding['attention_mask'].squeeze(),
            'labels': target_encoding['input_ids'].squeeze(),
        }

# Example Usage
texts = ["translate English to French: Hello world", "translate English to French: How are you"]
targets = ["Bonjour le monde", "Comment vas-tu?"]
dataset = MyDataset(tokenizer, texts, targets)
dataloader = DataLoader(dataset, batch_size=2)


In [5]:
from transformers import AdamW
epochs = 2
optimizer = AdamW(model.parameters(), lr=5e-5)

model.train()
for epoch in range(epochs):
  for batch in dataloader:
    optimizer.zero_grad()

    outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], labels=batch['labels'])
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch}, Loss: {loss.item()}")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 0, Loss: 17.157520294189453
Epoch 1, Loss: 15.467917442321777


In [6]:
model.save_pretrained('./fine-tuned_t5')
tokenizer.save_pretrained('./fine-tuned_t5')

('./fine-tuned_t5/tokenizer_config.json',
 './fine-tuned_t5/special_tokens_map.json',
 './fine-tuned_t5/spiece.model',
 './fine-tuned_t5/added_tokens.json')